In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import numpy as np

In [2]:
(train, val), info = tfds.load(
    "cats_vs_dogs",
    split=["train[:80%]", "train[80%:]"],
    with_info=True,
    as_supervised=True
)

In [3]:
def image_preprocessing(img, label):
    img = tf.image.resize(img, (resolution, resolution)) / 255.0
    return img, label

In [4]:
batch_size = 32
resolution = 224
num_examples = info.splits["train"].num_examples

train_batches = train.cache().shuffle(num_examples//4).map(image_preprocessing).batch(batch_size).prefetch(1)
val_batches = val.cache().map(image_preprocessing).batch(batch_size).prefetch(1)

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(resolution, resolution, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(2)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 128)       1

In [6]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [7]:
number_of_epochs = 15

history = model.fit(
    train_batches,
    epochs=number_of_epochs,
    validation_data=val_batches
)

Epoch 1/15
582/582 [==============================] - 63s 86ms/step - loss: 0.7081 - accuracy: 0.5195 - val_loss: 0.6762 - val_accuracy: 0.5752
Epoch 2/15
582/582 [==============================] - 91s 155ms/step - loss: 0.6779 - accuracy: 0.5729 - val_loss: 0.6593 - val_accuracy: 0.6096
Epoch 3/15
582/582 [==============================] - 40s 69ms/step - loss: 0.6573 - accuracy: 0.6063 - val_loss: 0.6159 - val_accuracy: 0.6853
Epoch 4/15
582/582 [==============================] - 28s 48ms/step - loss: 0.5963 - accuracy: 0.6828 - val_loss: 0.5221 - val_accuracy: 0.7425
Epoch 5/15
582/582 [==============================] - 28s 49ms/step - loss: 0.5069 - accuracy: 0.7503 - val_loss: 0.4581 - val_accuracy: 0.7885
Epoch 6/15
582/582 [==============================] - 28s 49ms/step - loss: 0.4482 - accuracy: 0.7933 - val_loss: 0.4195 - val_accuracy: 0.8113
Epoch 7/15
582/582 [==============================] - 28s 49ms/step - loss: 0.3957 - accuracy: 0.8250 - val_loss: 0.4015 - val_accuracy

In [8]:
url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(url, input_shape=(resolution, resolution, 3))

In [9]:
feature_extractor.trainable = False

In [10]:
model = tf.keras.models.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(2)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [11]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [12]:
number_of_epochs = 3

model.fit(
    train_batches,
    epochs=number_of_epochs,
    validation_data=val_batches
)

Epoch 1/3
582/582 [==============================] - 33s 50ms/step - loss: 0.1017 - accuracy: 0.9589 - val_loss: 0.0312 - val_accuracy: 0.9901
Epoch 2/3
582/582 [==============================] - 25s 44ms/step - loss: 0.0284 - accuracy: 0.9905 - val_loss: 0.0383 - val_accuracy: 0.9869
Epoch 3/3
582/582 [==============================] - 25s 42ms/step - loss: 0.0259 - accuracy: 0.9904 - val_loss: 0.0383 - val_accuracy: 0.9865


In [13]:
export_path = "./models/cats_vs_dogs_transfer_learning_v2.h5"
model.save(export_path)

In [14]:
load = tf.keras.models.load_model(
    export_path,
    custom_objects={"KerasLayer": hub.KerasLayer})

load.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [15]:
model_pred = model.predict(val_batches.take(1))

In [16]:
load_pred = load.predict(val_batches.take(1))

In [17]:
(abs(model_pred - load_pred)).max()

0.0